In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
import re
import matplotlib as mpl
from matplotlib import rc
import missingno as msno
import warnings
import datetime
warnings.filterwarnings('ignore')

# matplotlib 한글 사용을 위한 코드
rc('font', family='AppleGothic')

In [16]:
type(two_col['rating_audiences'][3])

str

In [10]:
two_col = pd.read_csv('edit.csv').drop(columns='Unnamed: 0')
two_col

,audience_rating,rating_audiences
0,8.612376613515565,5268
1,9.272090988626422,1143
2,8.96078431372549,918
3,8.337789203084833,1945
4,8.172131147540984,610
...,...,...
923,9.6,10
924,8.870588235294118,85
925,6.5,4
926,8.428571428571429,63


In [37]:
# for i, value in enumerate(final_df['audience_rating']):
#     if two_col['audience_rating'][i] != '-' and float(two_col['audience_rating'][i]) != float(value):
#         print(i)

In [8]:
final_df = pd.read_csv('final_df_06.csv').drop(columns='Unnamed: 0')

In [30]:
final_df['audience_rating'][2] = float(8.96078431372549)

In [40]:
final_df.to_csv('final_df_07.csv')

In [5]:
for i in range(0, len(movie_code_list)):
    
    # 먼저, 평점의 가장 마지막 페이지에 해당하는 url의 document구조를 가져온다
    code = int(movie_code_list[i])
    num = 99999
    url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=after&onlyActualPointYn=Y&onlySpoilerPointYn=N&order=newest&page={}'.format(code, num)
    
    # 현재 페이지 알아내기
    try:
        req = requests.get(url)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")
        number_list = response.xpath('/html/body/div/div/div[6]/div/a/span/text()').extract()
        last_number = number_list[len(number_list) - 1]

        cur_page = int(last_number.replace(',', ''))
        cur_page

        # 평가의 마지막 페이지 마지막 관객의 평점올린 날짜를 가져와서 7주일 이후의 날짜(limit_date) 만들기
        time_elements = response.xpath('/html/body/div/div/div[5]/ul/li/div/dl/dt/em[2]/text()')
        start_date = time_elements[len(time_elements) - 1].extract().split(' ')[0]
        start_date = start_date.split(" ")[0].split('.')
        limit_date = datetime.datetime(int(start_date[0]), int(start_date[1]), int(start_date[2])) + datetime.timedelta(weeks=1)

        audience_number = 0
        total_rating = 0
        flag = 0

        # 여기서부터 정확한 페이지로 크롤링 시작
        for page in range(cur_page, 0, -1):
            if flag == 1:
                break

            url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=after&onlyActualPointYn=Y&onlySpoilerPointYn=N&order=newest&page={}'.format(code, page)
            req = requests.get(url)
            response = TextResponse(req.url, body=req.text, encoding="utf-8")

            rating_elements = response.xpath('/html/body/div/div/div[5]/ul/li')

            # 평가의 해당 페이지에서, 각 평점 엘리먼트 하나마다 날짜를 확인하면서 limit을 넘겼는지 체크한다
            for n in range(len(rating_elements), 0, -1):
                star = response.xpath('/html/body/div/div/div[5]/ul/li[{}]/div[1]/em/text()'.format(n)).extract()[0]
                cur_date = response.xpath('/html/body/div/div/div[5]/ul/li[{}]/div[2]/dl/dt/em[2]/text()'.format(n)).extract()[0].split(' ')[0].split('.')

                check_limit = limit_date - datetime.datetime(int(cur_date[0]), int(cur_date[1]), int(cur_date[2]))
                if check_limit.days < 0:
                    flag = 1
                    break

                total_rating = total_rating + int(star)
                audience_number += 1

        rating = {'audience_rating': total_rating/audience_number}
        audience = {'rating_audiences': audience_number}

        audience_rating.append(rating)
        rating_audiences.append(audience)
        
    except Exception:
        rating = {'audience_rating': '-'}
        audience = {'rating_audiences': '-'}
        audience_rating.append(rating)
        rating_audiences.append(audience)
    

NameError: name 'movie_code_list' is not defined